# A notebook to try out different llm promts and models until the setup is complete. 

For the diffrent Prompts i will orient myself on the following simular Projects: 

* https://arxiv.org/pdf/2403.11996
* https://towardsdatascience.com/how-to-convert-any-text-into-a-graph-of-concepts-110844f22a1a
* https://towardsdatascience.com/text-to-knowledge-graph-made-easy-with-graph-maker-f3f890c0dbe8

Using the following model mistral:instruct using ollama.


``
pip install networkx matplotlib ollama pandas
``


### Test if the model is working



In [7]:
import ollama
# response = ollama.chat(model='mistral:instruct', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])

In [8]:
input = """
Graph-based methods are widely used in various fields including social network analysis, biology, and computer science. 
For instance, in biology, protein-protein interactions can be modeled as graphs where nodes represent proteins and edges 
represent interactions between them. Similarly, in social networks, nodes can represent individuals and edges represent 
relationships such as friendships or professional connections. Computer scientists often use graph algorithms to solve 
problems related to networking, data organization, and artificial intelligence.
"""



In [16]:
# Example Promt form: the first article 

SYS_PROMPT = (
    "You are a network graph maker who extracts terms and their relations from a given context. "
    "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
    "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
    "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
        "\tTerms may include object, entity, location, organization, person, \n"
        "\tcondition, acronym, documents, service, concept, etc.\n"
        "\tTerms should be as atomistic as possible\n\n"
    "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
        "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
        "\tTerms can be related to many other terms\n\n"
    "Thought 3: Find out the relation between each such related pair of terms. \n\n"
    "Format your output as a list of json. Each element of the list contains a pair of terms"
    "and the relation between them, like the follwing: \n"
    "[\n"
    "   {\n"
    '       "node_1": "A concept from extracted ontology",\n'
    '       "node_2": "A related concept from extracted ontology",\n'
    '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
    "   }, {...}\n"
    "]"
)

USER_PROMPT = f"context: ```{input}``` \n\n output: "

In [17]:
# Combine the system and user prompts
combined_prompt = SYS_PROMPT + USER_PROMPT
combined_prompt

'You are a network graph maker who extracts terms and their relations from a given context. You are provided with a context chunk (delimited by ```) Your task is to extract the ontology of terms mentioned in the given context. These terms should represent the key concepts as per the context. \nThought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n\tTerms may include object, entity, location, organization, person, \n\tcondition, acronym, documents, service, concept, etc.\n\tTerms should be as atomistic as possible\n\nThought 2: Think about how these terms can have one on one relation with other terms.\n\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n\tTerms can be related to many other terms\n\nThought 3: Find out the relation between each such related pair of terms. \n\nFormat your output as a list of json. Each element of the list contains a pair of termsand the relation between them, 

In [20]:
response = ollama.chat(model='mistral:instruct', messages=[
  {
    'role': 'user',
    'content': combined_prompt,
  },
])
# print(response['message']['content'])

# Set the output of the response to data
data = response['message']['content']
data

' [\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Social network analysis",\n    "edge": "Graph-based methods are widely used in social network analysis"\n  },\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Biology",\n    "edge": "Graph-based methods are widely used in Biology for modeling protein-protein interactions"\n  },\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Computer science",\n    "edge": "Graph-based methods are widely used in Computer Science"\n  },\n  {\n    "node_1": "Nodes",\n    "node_2": "Proteins",\n    "edge": "In Biology, nodes represent proteins"\n  },\n  {\n    "node_1": "Nodes",\n    "node_2": "Individuals",\n    "edge": "In Social Networks, nodes represent individuals"\n  },\n  {\n    "node_1": "Edges",\n    "node_2": "Interactions between proteins",\n    "edge": "Edges represent interactions between proteins in Biology"\n  },\n  {\n    "node_1": "Edges",\n    "node_2": "Friendships or professional connections",\n    "edg

In [33]:
data = response['message']['content']
data

' [\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Social network analysis",\n    "edge": "Graph-based methods are widely used in social network analysis"\n  },\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Biology",\n    "edge": "Graph-based methods are widely used in Biology for modeling protein-protein interactions"\n  },\n  {\n    "node_1": "Graph-based methods",\n    "node_2": "Computer science",\n    "edge": "Graph-based methods are widely used in Computer Science"\n  },\n  {\n    "node_1": "Nodes",\n    "node_2": "Proteins",\n    "edge": "In Biology, nodes represent proteins"\n  },\n  {\n    "node_1": "Nodes",\n    "node_2": "Individuals",\n    "edge": "In Social Networks, nodes represent individuals"\n  },\n  {\n    "node_1": "Edges",\n    "node_2": "Interactions between proteins",\n    "edge": "Edges represent interactions between proteins in Biology"\n  },\n  {\n    "node_1": "Edges",\n    "node_2": "Friendships or professional connections",\n    "edg

In [34]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import pandas as pd

# Parse the JSON string to a Python list of dictionaries
data = json.loads(data)


G = nx.Graph()

# Add nodes and edges to the graph
for relation in data:
    G.add_node(relation['node_1'])
    G.add_node(relation['node_2'])
    G.add_edge(relation['node_1'], relation['node_2'], label=relation['edge'])

# Create DataFrames for nodes and edges
nodes_df = pd.DataFrame(G.nodes(), columns=["Node"])
edges_df = pd.DataFrame([(u, v, d['label']) for u, v, d in G.edges(data=True)], columns=["Node_1", "Node_2", "Edge"])

# Print 
print("Nodes DataFrame:")
print(nodes_df)
print("\nEdges DataFrame:")
print(edges_df)

Nodes DataFrame:
                                      Node
0                      Graph-based methods
1                  Social network analysis
2                                  Biology
3                         Computer science
4                                    Nodes
5                                 Proteins
6                              Individuals
7                                    Edges
8            Interactions between proteins
9  Friendships or professional connections

Edges DataFrame:
                Node_1                                   Node_2  \
0  Graph-based methods                  Social network analysis   
1  Graph-based methods                                  Biology   
2  Graph-based methods                         Computer science   
3                Nodes                                 Proteins   
4                Nodes                              Individuals   
5                Edges            Interactions between proteins   
6                Edges 

In [38]:
from pyvis.network import Network

def visualize_graph_pyvis(G):
    net = Network(notebook=True)

    for node in G.nodes:
        net.add_node(node, label=node)

    for edge in G.edges(data=True):
        net.add_edge(edge[0], edge[1], title=edge[2]['label'])

    net.show("graph.html")

# Create a graph using the pyvis library
visualize_graph_pyvis(G)

graph.html
